<a href="https://www.kaggle.com/code/binfeng2021/what-is-bbpe-tokenizer-behind-llms?scriptVersionId=196252643" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Project Introduction <a class="anchor"  id="Intro"></a>

Unless you have been living under a rock, I am sure that you have heard, used or even worked on some generative AIs. I have been amazed by the ChatGPT when it was first realsed in 2022. I have been wondering how this thing works, it feels like some dark magic. Over the years, this new AI hype has never trended down, but it has been recognized as the next economic engine for decades to come. 

If you are someone like me, you might find it exciting to persue a deeper technical understanding of generative AI. This is a huge topic and there are so many different concepts that worth to be studyed and researched. So we will only focus on one concept in this post: the tokenization behind most of the state-of-the-art large langauge models (LLMs). 

For many LLMs, the quality of the tokenization determines the quality of the training data fed into the neuron network. In many cases, the root causes of many LLMs' issues are related to the tokenization used. So it is important to build a solid konwledge fundation on this topic if we want to further puesue a deeper understanding on the LLMs and other generative AIs. 

# Objective <a class="anchor"  id="objective"></a>

BBPE (Byte Level Byte Pair Encoding) tokenizer was brought into public's attention by OpenAI in their GPT2 paper. It gains more popularity afterwards and it now becomes the standard parctices in the start of art LLMs. So this post, we will go through the following sections that aims to understand how BBPE works, and why they are popular compared to other type of tokenizers. 

**Table of Contents**
* [Project Introduction](#Intro)
* [Objective](#objective)
* [Why BBPE?](#chapter1)
    * [Word-level Tokenizer](#section_1_1)
    * [Character-level Tokenizer](#section_1_2)
    * [Byte-level Tokenizer](#section_1_3)
    * [Advantage of BBPE Tokenizer](#section_1_4)
* [Deep dive into BBPE (Byte-level Byte Pair Encoding) Tokenizer](#chapter2)
    * [Quick intro to BPE](#section_2_1)
    * [BBPE by steps](#section_2_2)
        * [Convert texts to raw bytes](#section_2_2_1)
        * [Find the most frequent pair in the whole sequence](#section_2_2_2)
        * [replace the most frequent pair with new ID](#section_2_2_3)
        * [Train a BBPE tokenizer with all steps above](#section_2_2_4)
        * [Encoding and Decoding ](#section_2_2_5)
* [Conclusion](#chapter3)

# Why BBPE? <a class="anchor"  id="chapter1"></a>

To understand why BBPE could be a popular choice as the tokenizer in many LLMs, we need to get faimilar with other types of tokenizer existed and what the disadvantages of them compared to the BBPE. 

In this section, we will look into different type of tokenizers and explain why other types are not good fit for the large language models.

* Word-level Tokenizer
* Character-level Tokenizer
* Byte-level Tokenizer

## Word-level Tokenizer <a class="anchor"  id="section_1_1"></a>

Word-level tokenizer will divide a sentense into words or sub-words based on certain rules. For example, spliting a sentence "Dogs are cute." with the spaces between and we can get ["Dogs", "are", "cute."]. And then each of those words or sub-words can be encoded into integers which can be used for model training. 

Let's go through a example and discuss the potential issue with this when training a large language model with massive amount of corpus. 

In [1]:
# 1. word level tokenization
import spacy
word_tokenizer = spacy.load("en_core_web_sm")

In [2]:
eng_text = 'Hello, how are you doing today👋?'
print(list(word_tokenizer(eng_text)))
print(f"number of tokens encoded: {len(word_tokenizer(eng_text))}")

[Hello, ,, how, are, you, doing, today, 👋, ?]
number of tokens encoded: 9


In [3]:
chinese_text = '你好，你今天过的怎么样👋？'

print(list(word_tokenizer(chinese_text))) 
print(f"number of tokens encoded: {len(word_tokenizer(chinese_text))}")

[你好，你今天过的怎么样, 👋, ？]
number of tokens encoded: 3


For the first English text, the word tokenizer works pretty well. It recognized every word and seperate the text into small sections. 

However, when I provide the Chinese version of the text, the tokenizer could not recognize the Chinese characters and we have all Chinese characters combined into one token.

The issue is that we only imported the English vocabulary, so the tokenizer cannot recognize other language. In order to create a tokenizer that can work for all languages around the world, we will have to include all possible words in the vocabulary. Not only that, we will have to include all different version of the same characters, such as "cat", "cats", "Cat"...

We can only image how big the size of the vocabulary will be for word-level tokenizer. Just to give a perspective, TransformerXL is a word-level tokenizer and its vocabulary size is 267,735.

## Character-level Tokenizer <a class="anchor"  id="section_1_2"></a>

Other than word level, another common approach is to use Unicode to tokenize text in character level. This type of tokenizer will split the texts into individual characters. For the same example "Dogs are cute.", we can split it into a list of characters ["D", "o", "g", "s", " ", "a", "r", "e", " ", "c", "u", "t", "e", "."]. 

In this space, using Unicode is a popular choice because Unicode is a text encoding standard designed to support all writen characters acorss all major languages. We will go through a example using character level tokenizer and we can discuss why they are also not the best option for LLMs. 

In [4]:
# same example with the first text:
print(f"All characters in the English text: {[*eng_text]}\n")
print(f"Tokens in Unicode: {[ord(x) for x in eng_text]}\n")
print(f"number of tokens encoded: {len(eng_text)}")

All characters in the English text: ['H', 'e', 'l', 'l', 'o', ',', ' ', 'h', 'o', 'w', ' ', 'a', 'r', 'e', ' ', 'y', 'o', 'u', ' ', 'd', 'o', 'i', 'n', 'g', ' ', 't', 'o', 'd', 'a', 'y', '👋', '?']

Tokens in Unicode: [72, 101, 108, 108, 111, 44, 32, 104, 111, 119, 32, 97, 114, 101, 32, 121, 111, 117, 32, 100, 111, 105, 110, 103, 32, 116, 111, 100, 97, 121, 128075, 63]

number of tokens encoded: 32


In [5]:
print(f"All characters in the Chinese text: {[*chinese_text]}\n")
print(f"Tokens in Unicode: {[ord(x) for x in chinese_text]}\n")
print(f"number of tokens encoded: {len(chinese_text)}")

All characters in the Chinese text: ['你', '好', '，', '你', '今', '天', '过', '的', '怎', '么', '样', '👋', '？']

Tokens in Unicode: [20320, 22909, 65292, 20320, 20170, 22825, 36807, 30340, 24590, 20040, 26679, 128075, 65311]

number of tokens encoded: 13


The Unicode tokenizer seems to work very well for our examples. Since Unicode contains all characters in all languages, the tokenizer is able to conver all texts into different tokens. Notice that the unicode can convert the texts into a list of integers, so in theory, LLMs can use those to train the neurons, but what is the catch? 

Here are two disadvantages of using this character-level tokenizer naively.

1. Unicode contains almost 150K different characters, so the base vocabulary size will be quite large. If we then further combine multiple characters into new tokens, the size of vocabulary will grow even more.

2. Unicode is a constantly envolving standard. If we use the unicode as our tokenizer, we will have new characters that our old version of tokenizer cannot recognize.

## Byte-level Tokenizer <a class="anchor"  id="section_1_3"></a>

One of the approach to limit the vocabulary size it to use byte level tokenizer. When the tokenizer is byte level. For example, we can encode the unicode sequences we obtained prior to UTF-8, UTF-16 or UTF-32 to obtain byte-level tokens. Since each byte is 8-bits binary numbers, so there are total 2^8 different possibilities for one byte which equals to 256 as vocabulary size.

Let's use the English texts as example in following to explore different UTF encoding standards:

In [6]:
print(f"byte level of tokens of the text in utf-8: {list(chinese_text.encode('utf-8'))}\n")
print(f"number of tokens encoded: {len(list(chinese_text.encode('utf-8')))}")

byte level of tokens of the text in utf-8: [228, 189, 160, 229, 165, 189, 239, 188, 140, 228, 189, 160, 228, 187, 138, 229, 164, 169, 232, 191, 135, 231, 154, 132, 230, 128, 142, 228, 185, 136, 230, 160, 183, 240, 159, 145, 139, 239, 188, 159]

number of tokens encoded: 40


### UTF-8

In [7]:
# use unicode to represents all characters and then encode it using different UTF standards
print(f"byte level of tokens of the text in utf-8: {list(eng_text.encode('utf-8'))}\n")
print(f"number of tokens encoded: {len(list(eng_text.encode('utf-8')))}")

byte level of tokens of the text in utf-8: [72, 101, 108, 108, 111, 44, 32, 104, 111, 119, 32, 97, 114, 101, 32, 121, 111, 117, 32, 100, 111, 105, 110, 103, 32, 116, 111, 100, 97, 121, 240, 159, 145, 139, 63]

number of tokens encoded: 35


### UTF-16

In [8]:
# use unicode to represents all characters and then encode it using different UTF standards
print(f"byte level of tokens of the text in utf-16: {list(eng_text.encode('utf-16'))}\n")
print(f"number of tokens encoded: {len(list(eng_text.encode('utf-16')))}")

byte level of tokens of the text in utf-16: [255, 254, 72, 0, 101, 0, 108, 0, 108, 0, 111, 0, 44, 0, 32, 0, 104, 0, 111, 0, 119, 0, 32, 0, 97, 0, 114, 0, 101, 0, 32, 0, 121, 0, 111, 0, 117, 0, 32, 0, 100, 0, 111, 0, 105, 0, 110, 0, 103, 0, 32, 0, 116, 0, 111, 0, 100, 0, 97, 0, 121, 0, 61, 216, 75, 220, 63, 0]

number of tokens encoded: 68


### UTF-32

In [9]:
# use unicode to represents all characters and then encode it using different UTF standards
print(f"byte level of tokens of the text in utf-32: {list(eng_text.encode('utf-32'))}\n")
print(f"number of tokens encoded: {len(list(eng_text.encode('utf-32')))}")

byte level of tokens of the text in utf-32: [255, 254, 0, 0, 72, 0, 0, 0, 101, 0, 0, 0, 108, 0, 0, 0, 108, 0, 0, 0, 111, 0, 0, 0, 44, 0, 0, 0, 32, 0, 0, 0, 104, 0, 0, 0, 111, 0, 0, 0, 119, 0, 0, 0, 32, 0, 0, 0, 97, 0, 0, 0, 114, 0, 0, 0, 101, 0, 0, 0, 32, 0, 0, 0, 121, 0, 0, 0, 111, 0, 0, 0, 117, 0, 0, 0, 32, 0, 0, 0, 100, 0, 0, 0, 111, 0, 0, 0, 105, 0, 0, 0, 110, 0, 0, 0, 103, 0, 0, 0, 32, 0, 0, 0, 116, 0, 0, 0, 111, 0, 0, 0, 100, 0, 0, 0, 97, 0, 0, 0, 121, 0, 0, 0, 75, 244, 1, 0, 63, 0, 0, 0]

number of tokens encoded: 132


Notice that the number of tokens doubles everytime when we switch from UTF-8, UFT-16 and UTF-32. There is advantage of using UTF-32 as encoder since all characters will be encoded to the same length (4 Bytes). But we see there are so many zeros in the encoded tokens, and that is a big waste in memory. 

Therefore, in current standard practice, UTF-8 is the best choice because it is variable length encoder (1-4 bytes), so it can minimize the number of tokens in the end.

### Issues of using unicode naively

As we can see from above, when we use UTF-8 to encode our text to byte level, we can obtain a nice list of tokens that can be fed into neuron networks. However, there is a big disadvantage of using that naively as our tokenizer. 

Since for each byte, there is only 256 possibilities. For a complicated characters, it will be encoded into multiple bytes. Shown below.

In [10]:
print(f"English character 'a' is encoded as {list('a'.encode('utf-8'))}")
print(f"Chinese character '龙' is encoded as {list('龙'.encode('utf-8'))}")

English character 'a' is encoded as [97]
Chinese character '龙' is encoded as [233, 190, 153]


When using UTF-8 to encode our texts into tokens, we will expect a very long sequence of tokens as the trade off of having small base vocabulary. However, transformers used in LLMs can only support certain length of input because transformer has limited attention length in the current level of computation. Having a long sequence will limit the efficiency of the training of transformer. 

Therefore, a text compression is needed in this case to reduce the token size. Also, we want to adjust the vocabulary size so we can further balance between the vocabulary size and the encoded tokens size. 

## Advantages of BBPE tokenizer <a class="anchor"  id="section_1_4"></a>

The idea of using BBPE tokenizer over other kinds are menioned in the GPT2 paper. It aims to solve the issues that we have mentioned above. 

- First, using byte level tokenizer, we can have a base vocabulary of 256 which is very small size. 
- Second, by applying BPE algorithm, we can also add new merge rules to our vocabulary to expand the size, so the encoded sequence can be compressed. 

So by applying BPE in the byte level is a clever way to give us the flexibility of choosing a vocabulary size that can best fit our needs. In the next section, I will go deeper into this tokenizer approach and explain how it works.

# Deep dive into BBPE (Byte-level Byte Pair Encoding) Tokenizer <a class="anchor"  id="chapter2"></a>

## Quick intro to BPE <a class="anchor"  id="section_2_1"></a>

Byte Pair Encoding is a text compression technique that merges commonly appeared character pairs. Here is a quick example (in Wikipedia):

1. Given data for encode: **aaabdaaabac**

2. Since byte pair 'aa' are the most often occurred, we will create a merge rule by replace all this pair with a new byte that have not included in the data: **Z = aa**. Now, the data becomes: **ZabdZabac**

3. Repeat step 2: found the most frequent appearred pair as 'ab', create merge rule: **Y = ab**. Now, the data is **ZYdZYac**.

4. Repeat again with new merge rule: **X = ZY**. And data is now **XdXac**. 

5. There is no pair that occurs more than once in the data. So we will have a compressed version of the data: **XdXac** and we can decode it back to orginal with the three merge rules: **X=ZY; Y=ab; Z=aa**


## BBPE by steps <a class="anchor"  id="section_2_2"></a>
When BPE is operated on byte level, the given text will be encoded in UTF-8 into raw bytes. Next the raw bytes will be used to create merge rules and we will then use the merge rule to compress the previous bytes sequence. After a few iterations, the initial sequences will have a much smaller size. Let's go through the training process one step at a time:

#### 1. Convert texts to raw bytes <a class="anchor"  id="section_2_2_1"></a>

In [11]:
# I took a random paragraph from the BPE Wikipedia page as our example text.
example_text = "The original algorithm operates by iteratively replacing the most common contiguous sequences of characters in a target text with unused 'placeholder' bytes. The iteration ends when no sequences can be found, leaving the target text effectively compressed. Decompression can be performed by reversing this process, querying known placeholder terms against their corresponding denoted sequence, using a lookup table. In the original paper, this lookup table is encoded and stored alongside the compressed text."

In [12]:
tokens_in_utf8 = list(example_text.encode("utf-8"))
print(f"Using UTF-8 to encode the text to raw bytes: {tokens_in_utf8}\n")
print(f"the number of tokens is {len(tokens_in_utf8)}")

Using UTF-8 to encode the text to raw bytes: [84, 104, 101, 32, 111, 114, 105, 103, 105, 110, 97, 108, 32, 97, 108, 103, 111, 114, 105, 116, 104, 109, 32, 111, 112, 101, 114, 97, 116, 101, 115, 32, 98, 121, 32, 105, 116, 101, 114, 97, 116, 105, 118, 101, 108, 121, 32, 114, 101, 112, 108, 97, 99, 105, 110, 103, 32, 116, 104, 101, 32, 109, 111, 115, 116, 32, 99, 111, 109, 109, 111, 110, 32, 99, 111, 110, 116, 105, 103, 117, 111, 117, 115, 32, 115, 101, 113, 117, 101, 110, 99, 101, 115, 32, 111, 102, 32, 99, 104, 97, 114, 97, 99, 116, 101, 114, 115, 32, 105, 110, 32, 97, 32, 116, 97, 114, 103, 101, 116, 32, 116, 101, 120, 116, 32, 119, 105, 116, 104, 32, 117, 110, 117, 115, 101, 100, 32, 39, 112, 108, 97, 99, 101, 104, 111, 108, 100, 101, 114, 39, 32, 98, 121, 116, 101, 115, 46, 32, 84, 104, 101, 32, 105, 116, 101, 114, 97, 116, 105, 111, 110, 32, 101, 110, 100, 115, 32, 119, 104, 101, 110, 32, 110, 111, 32, 115, 101, 113, 117, 101, 110, 99, 101, 115, 32, 99, 97, 110, 32, 98, 101, 32, 102

#### 2. find the most frequent pair in the whole sequence <a class="anchor"  id="section_2_2_2"></a>

In [13]:
# First, count the pair frequencies
count = {}
for i in zip(tokens_in_utf8, tokens_in_utf8[1:]):
    if i not in count.keys():
        count[i] = 1
    else:
        count[i] += 1

# sort the pair counts to find the most frequent pair
sorted_pair_counts = {k: v for k, v in sorted(count.items(), key = lambda item: item[1], reverse = True)}
print(f"we will sort the pair counts in order to find the most frequent pair to merge: \n{sorted_pair_counts}\n")
print(f"the most frequently appearred pair is {list(sorted_pair_counts.keys())[0]}")

we will sort the pair counts in order to find the most frequent pair to merge: 
{(32, 116): 15, (101, 114): 11, (101, 32): 10, (105, 110): 10, (116, 101): 10, (115, 32): 10, (116, 104): 9, (101, 115): 9, (110, 32): 9, (104, 101): 8, (32, 99): 8, (114, 101): 7, (110, 103): 7, (99, 111): 7, (101, 110): 7, (101, 100): 7, (100, 32): 7, (111, 114): 6, (32, 97): 6, (103, 32): 6, (116, 32): 6, (111, 110): 6, (115, 101): 6, (99, 101): 6, (32, 98): 5, (100, 101): 5, (32, 111): 4, (97, 108): 4, (105, 116): 4, (114, 97): 4, (121, 32): 4, (32, 105): 4, (116, 105): 4, (97, 99): 4, (111, 109): 4, (32, 115): 4, (113, 117): 4, (117, 101): 4, (110, 99): 4, (116, 97): 4, (110, 100): 4, (44, 32): 4, (112, 114): 4, (115, 115): 4, (115, 105): 4, (32, 112): 4, (114, 105): 3, (105, 103): 3, (112, 101): 3, (97, 116): 3, (98, 121): 3, (118, 101): 3, (112, 108): 3, (108, 97): 3, (115, 116): 3, (117, 115): 3, (101, 113): 3, (97, 114): 3, (101, 120): 3, (120, 116): 3, (46, 32): 3, (32, 101): 3, (110, 111): 3, (97

#### 3. replace the most frequent pair with new ID <a class="anchor"  id="section_2_2_3"></a>

In [14]:
# since we are create our first merge rule, we can set the new id as 256
merge_pair = list(sorted_pair_counts.keys())[0]
new_tokens = []
new_id = 256
i = 0

while i < len(tokens_in_utf8):
    # if we are at the last index, we will break out the loop to aviod error
    # then we will match current pair with the merge pair
    if (i < len(tokens_in_utf8) - 1) and (tokens_in_utf8[i] == merge_pair[0]) and (tokens_in_utf8[i+1] == merge_pair[1]):
        new_tokens.append(new_id)
        # if we found a merge pair, add 2 to i to skip next value in list
        i += 2
    else:
        new_tokens.append(tokens_in_utf8[i])
        i += 1

In [15]:
print(f"After one iteration, we got our new list of tokens: {new_tokens}\n")
print(f"The new token size is {len(new_tokens)}")

After one iteration, we got our new list of tokens: [84, 104, 101, 32, 111, 114, 105, 103, 105, 110, 97, 108, 32, 97, 108, 103, 111, 114, 105, 116, 104, 109, 32, 111, 112, 101, 114, 97, 116, 101, 115, 32, 98, 121, 32, 105, 116, 101, 114, 97, 116, 105, 118, 101, 108, 121, 32, 114, 101, 112, 108, 97, 99, 105, 110, 103, 256, 104, 101, 32, 109, 111, 115, 116, 32, 99, 111, 109, 109, 111, 110, 32, 99, 111, 110, 116, 105, 103, 117, 111, 117, 115, 32, 115, 101, 113, 117, 101, 110, 99, 101, 115, 32, 111, 102, 32, 99, 104, 97, 114, 97, 99, 116, 101, 114, 115, 32, 105, 110, 32, 97, 256, 97, 114, 103, 101, 116, 256, 101, 120, 116, 32, 119, 105, 116, 104, 32, 117, 110, 117, 115, 101, 100, 32, 39, 112, 108, 97, 99, 101, 104, 111, 108, 100, 101, 114, 39, 32, 98, 121, 116, 101, 115, 46, 32, 84, 104, 101, 32, 105, 116, 101, 114, 97, 116, 105, 111, 110, 32, 101, 110, 100, 115, 32, 119, 104, 101, 110, 32, 110, 111, 32, 115, 101, 113, 117, 101, 110, 99, 101, 115, 32, 99, 97, 110, 32, 98, 101, 32, 102, 111

We can notice that with one iteration of BPE, we reduced our token sizes from 509 to 494. This makes sense because we have merge rule (32, 116) -> 256 which have appearred 15 times. 

**Note** 

In the previous three steps, we have seen how BBPE works in slow motion. Hopefully you can see why BBPE tokenizer is a good choice for many LLMs now. Here are some questions I had prior and I want to make sure they are clear for anyone who might still have confusions:

1. Why BBPE tokenizer is able to recognize all characters? Because all characters will be encoded into raw bytes using UTF-8 first. Even the tokenizer has never encoutered a character in the training set, it can still encode it with the raw bytes. The character could be represented with multiple bytes, but it can still be encoded. 

2. Why BBPE only have 256 vocabulary size? BBPE tokenizer has a flexible vocabulary size, but its base vocabulary size is 256 because a raw byte consists of eight bits, and there are total 2^8 possibilities of one byte. However, when the BBPE tokenizer is trained, as new merge rules created, the vocabulary size will grow. 

3. Why BBPE tokenizer is a more efficient choice comparing to using Unicode? As we have shown above, because of the merge rules, we can compress a combination of tokens by replacing them with one new token. So we will have a shorter token sequence to represent the same text compared to using Unicode naively. 

In the next section, we will combine the steps above and train our own BBPE tokenizer. 

#### 4. Train a BBPE tokenizer with all steps above <a class="anchor"  id="section_2_2_4"></a>

In [16]:
def count_pair_freq(tokens):
    count = {}
    for i in zip(tokens, tokens[1:]):
        if i not in count.keys():
            count[i] = 1
        else:
            count[i] += 1

    # sort the pair counts to find the most frequent pair
    sorted_pair_counts = {k: v for k, v in sorted(count.items(), key = lambda item: item[1], reverse = True)}
    return sorted_pair_counts

def merge(merge_pair, tokens, new_id):
    new_tokens = []
    i = 0

    while i < len(tokens):
        # if we are at the last index, we will break out the loop to aviod out of index error
        # then we will match current pair with the merge pair
        if (i < len(tokens) - 1) and (tokens[i] == merge_pair[0]) and (tokens[i+1] == merge_pair[1]):
            new_tokens.append(new_id)
            # if we found a merge pair, add 2 to i to skip next value in list
            i += 2
        else:
            new_tokens.append(tokens[i])
            i += 1
    return new_tokens

In [17]:
# as mentioned, with BBPE, we can adjust the size of the final vocabulary by defining the number of merges
given_texts = "Byte pair encoding[1][2] (also known as digram coding)[3] is an algorithm, first described in 1994 by Philip Gage for encoding strings of text into tabular form for use in downstream modeling.[4] Its modification is notable as the large language model tokenizer with an ability to combine both tokens that encode single characters (including single digits or single punctuation marks) and those that encode whole words (even the longest compound words).[5][6][7] This modification, in the first step, assumes all unique characters to be an initial set of 1-character long n-grams (i.e. initial 'tokens'). Then, successively the most frequent pair of adjacent characters is merged into a new, 2-character long n-gram and all instances of the pair are replaced by this new token. This is repeated until a vocabulary of prescribed size is obtained. Note that new words can always be constructed from final vocabulary tokens and initial-set characters.[8] All the unique tokens found in a corpus are listed in a token vocabulary, the size of which, in the case of GPT-3.5 and GPT-4, is 100256. The difference between the modified and the original algorithm is that the original algorithm does not merge the most frequent pair of bytes of data, but replaces them by a new byte that was not contained in the initial dataset. A lookup table of the replacements is required to rebuild the initial dataset. The algorithm is effective for tokenization because it has low computational overhead and remains consistent and reliable."
tokens = given_texts.encode('utf-8')
# convert the binary tokens to a list of integers for each visualization
tokens = list(map(int, tokens))
print(f"the intial token size before any compression is {len(tokens)}")

the intial token size before any compression is 1520


In [18]:
# perform BBPE 
desired_vocab_size = 280
num_of_merges = desired_vocab_size - 256
# we need to create a dictionary to store all merge rule, it will be used for decoding purpose
merge_rule = {}

for i in range(num_of_merges):
    pair_counts = count_pair_freq(tokens)
    # we will take the highest rank pair as our merge pair
    merge_pair = list(pair_counts.keys())[0]
    new_id = 256 + i
    merge_rule[merge_pair] = new_id
    print(f"Merge pair {merge_pair} and replace with new id {new_id}")
    tokens = merge(merge_pair, tokens, new_id)

print(f"\nWith {len(merge_rule)} merge rules, we obtain new tokens size of {len(tokens)}")

Merge pair (101, 32) and replace with new id 256
Merge pair (115, 32) and replace with new id 257
Merge pair (105, 110) and replace with new id 258
Merge pair (116, 104) and replace with new id 259
Merge pair (32, 97) and replace with new id 260
Merge pair (101, 110) and replace with new id 261
Merge pair (116, 32) and replace with new id 262
Merge pair (100, 32) and replace with new id 263
Merge pair (111, 114) and replace with new id 264
Merge pair (259, 256) and replace with new id 265
Merge pair (116, 111) and replace with new id 266
Merge pair (97, 114) and replace with new id 267
Merge pair (114, 101) and replace with new id 268
Merge pair (97, 108) and replace with new id 269
Merge pair (105, 257) and replace with new id 270
Merge pair (116, 101) and replace with new id 271
Merge pair (116, 105) and replace with new id 272
Merge pair (99, 111) and replace with new id 273
Merge pair (111, 102) and replace with new id 274
Merge pair (116, 97) and replace with new id 275
Merge pair

In [19]:
# first, let's create the vocabulary list: it contains two part: base vocabulary and the merged rules
# construct base vocab first, which are the raw bytes of each integer from 0 to 255
vocabulary = {integer: bytes([integer]) for integer in range(256)}

# then, we will add all merge rules into the vocabulary
for k, v in merge_rule.items():
    # remember merge rule item has this format (token1, token2): new_token
    # and token1 and token2 are two integers, and new_token is a new integer outside of (0, 255)
    # we will add the rule into vocabulary in this format: new_token: bytes(token1) + bytes(token2)
    # since bytes(token1) and bytes(token2) are already stored in the dictionary, we can add the merge rule like follow:
    vocabulary[v] = vocabulary[k[0]] + vocabulary[k[1]]

print(f"Let's take a look at the obtained vocabulary: {vocabulary}")

Let's take a look at the obtained vocabulary: {0: b'\x00', 1: b'\x01', 2: b'\x02', 3: b'\x03', 4: b'\x04', 5: b'\x05', 6: b'\x06', 7: b'\x07', 8: b'\x08', 9: b'\t', 10: b'\n', 11: b'\x0b', 12: b'\x0c', 13: b'\r', 14: b'\x0e', 15: b'\x0f', 16: b'\x10', 17: b'\x11', 18: b'\x12', 19: b'\x13', 20: b'\x14', 21: b'\x15', 22: b'\x16', 23: b'\x17', 24: b'\x18', 25: b'\x19', 26: b'\x1a', 27: b'\x1b', 28: b'\x1c', 29: b'\x1d', 30: b'\x1e', 31: b'\x1f', 32: b' ', 33: b'!', 34: b'"', 35: b'#', 36: b'$', 37: b'%', 38: b'&', 39: b"'", 40: b'(', 41: b')', 42: b'*', 43: b'+', 44: b',', 45: b'-', 46: b'.', 47: b'/', 48: b'0', 49: b'1', 50: b'2', 51: b'3', 52: b'4', 53: b'5', 54: b'6', 55: b'7', 56: b'8', 57: b'9', 58: b':', 59: b';', 60: b'<', 61: b'=', 62: b'>', 63: b'?', 64: b'@', 65: b'A', 66: b'B', 67: b'C', 68: b'D', 69: b'E', 70: b'F', 71: b'G', 72: b'H', 73: b'I', 74: b'J', 75: b'K', 76: b'L', 77: b'M', 78: b'N', 79: b'O', 80: b'P', 81: b'Q', 82: b'R', 83: b'S', 84: b'T', 85: b'U', 86: b'V', 87:

#### 5. Encoding and Decoding <a class="anchor"  id="section_2_2_5"></a>

In step 4, we have trained our BBPE tokenizer and obtained our new vocabulary and merge rules. With those two dicitonary, it can be used to encode and decode between raw unicode bytes and tokens. In step 5, we will create encode and decode function to complete the functionality of this tokenizer.

In [20]:
def decoder(tokens, vocabulary):
    # this function will decode a list of integer tokens back to string
    raw_bytes = b"".join(vocabulary[token] for token in tokens)
    texts = raw_bytes.decode('utf-8')
    return texts

In [21]:
def encoder(text, merge_rule):
    # this function will encode a string into a list of tokens
    # first, use utf8 to encode the text into raw bytes
    tokens = list(text.encode("utf-8"))
    # second, based on merge rules to merge raw bytes
    # A good tip learned from the video: we can reuse the pair count function to get all possible pairs
    # then we can check if any of those pairs are in the merge rules to be more efficient
    pair_counts = count_pair_freq(tokens)
    # go through the rules from the first to last
    for merge_pair, new_id in merge_rule.items():
        # when we found the qualified pairs, then perform merge
        if merge_pair in pair_counts.keys():
            tokens = merge(merge_pair, tokens, new_id)
    return tokens

In [22]:
# to test the encoder and decoder, we can provide a text, use encoder and decoder together and we should get the iinitial text back
print(f"here is the initial texts provided: \n{given_texts}")
print('---')

returned_texts = decoder(encoder(given_texts, merge_rule), vocabulary)
print(f"here is the returned texts after running encoder and then decoder: \n{returned_texts}")

here is the initial texts provided: 
Byte pair encoding[1][2] (also known as digram coding)[3] is an algorithm, first described in 1994 by Philip Gage for encoding strings of text into tabular form for use in downstream modeling.[4] Its modification is notable as the large language model tokenizer with an ability to combine both tokens that encode single characters (including single digits or single punctuation marks) and those that encode whole words (even the longest compound words).[5][6][7] This modification, in the first step, assumes all unique characters to be an initial set of 1-character long n-grams (i.e. initial 'tokens'). Then, successively the most frequent pair of adjacent characters is merged into a new, 2-character long n-gram and all instances of the pair are replaced by this new token. This is repeated until a vocabulary of prescribed size is obtained. Note that new words can always be constructed from final vocabulary tokens and initial-set characters.[8] All the uni

In [23]:
given_texts == returned_texts

True

Great! Now we have trained our own BBPE tokenizer, and also built a encoder and decoder function for the trained tokenizer. 

# Conclusion <a class="anchor"  id="chapter3"></a>

In this post, we have go through the BBPE tokenizer with many details. Let's quickly recap what we have done so far:

1. In order to understand why BBPE tokenizer is a popular choice for LLMs, we have checked other types of tokenizer, such as word-level tokenizer, character-level tokenizer, or byte level using UTF-8 directly. All of those tokenizers have its own advantage, but when using in the LLMs task, they all lack some key factors. 

2. On the other hand, BBPE tokenizer starts with a very small base vocabulary, but also provide the flexibility to add new vocabulary by merging frequently appearred pairs. So the LLMs research can find the good balance between good size of vocabulary and good size of encoded tokens. This is the one advantage BBPE tokenizer can provide while other methods lack. 

3. Then, we went through the initial BPE algorithm, and the byte-level implementation steps by steps. To train a BBPE tokenizer, all training texts will be encoded into raw bytes first (normally in UTF-8). Next, we will perform BPE algorithm to find the most frequently appearred pairs to merge. We will repeat this step until we reached the desired vocabulary size. Finally, we will create final vocabulary dictionay and all merge rules. Those two things will be stored for future encode and decode tasks. 

If you enjoy reading this and find it helpful, please give me a upvote. If you have any suggesstion or questions, please leave a comment. Thank you so much for reading!

**Special Thanks to Andrej Karpathy's amazing work!**

If you have time and want to know more about BBPE, please watch Andrej Karpathy's video. He went even deeper on this topic and shared many thoughts and addtional information. The video link is shared below:

[Let's build the GPT Tokenizer](https://www.youtube.com/watch?v=zduSFxRajkE&t=108s)